# Interactive Data Visualization with Geographical Coordinates using Bokeh

In [1]:
# import library
import numpy as np
import pandas as pd

from bokeh.plotting import figure
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
from bokeh.palettes import RdYlBu5

## Part 1: Geographical data of counties in NC

In [2]:
# read geographical data
df = pd.read_csv('geo_bounary_NC.csv')
df.head()

,County Name,State-County,state abbr,State Abbr.,geometry,value,GEO_ID,GEO_ID2,Geographic Name,STATE num,COUNTY num,FIPS formula,Has error
0,Alamance,NC-Alamance,nc,NC,<Polygon><outerBoundaryIs><LinearRing><coordin...,670.628581,05000US37001,37001,"Alamance County, North Carolina",37,1,37001.0,NaN
1,Alexander,NC-Alexander,nc,NC,<Polygon><outerBoundaryIs><LinearRing><coordin...,265.535507,05000US37003,37003,"Alexander County, North Carolina",37,3,37003.0,NaN
2,Alleghany,NC-Alleghany,nc,NC,<Polygon><outerBoundaryIs><LinearRing><coordin...,751.976713,05000US37005,37005,"Alleghany County, North Carolina",37,5,37005.0,NaN
3,Anson,NC-Anson,nc,NC,<Polygon><outerBoundaryIs><LinearRing><coordin...,143.197884,05000US37007,37007,"Anson County, North Carolina",37,7,37007.0,NaN
4,Ashe,NC-Ashe,nc,NC,<Polygon><outerBoundaryIs><LinearRing><coordin...,289.695601,05000US37009,37009,"Ashe County, North Carolina",37,9,37009.0,NaN


### Data cleaning
Let's take a look at the first observation (row) and fifth column (geometry), which is a set of **longitude (x)** and **latitude (y)** of the boundary of county **Alamance**.

In [3]:
# take a look at the first row in geometry column
df.geometry[0]

'<Polygon><outerBoundaryIs><LinearRing><coordinates>-79.54207,35.88714 -79.54208,35.8894 -79.54203,35.8936 -79.54201,35.89584 -79.54198,35.89963 -79.54179,35.90126 -79.54146,35.91252 -79.54135,35.91924 -79.54126,35.92853 -79.54075,35.94736 -79.54033,35.96957 -79.53974,36.00014 -79.53963,36.01331 -79.53959,36.01881 -79.53943,36.03855 -79.53934,36.04565 -79.53931,36.05206 -79.53914,36.06962 -79.53894,36.08174 -79.53865,36.0945 -79.5385,36.10022 -79.53825,36.10733 -79.53794,36.11626 -79.53759,36.12066 -79.53725,36.13776 -79.53657,36.16036 -79.5363,36.16546 -79.53544,36.18487 -79.53468,36.19886 -79.53345,36.22022 -79.53239,36.24092 -79.53236,36.24134 -79.53233,36.24143 -79.53232,36.24155 -79.53221,36.24495 -79.5321,36.24847 -79.5321,36.24853 -79.5321,36.24858 -79.53043,36.24883 -79.52341,36.24985 -79.50911,36.24965 -79.49949,36.24954 -79.48785,36.24919 -79.4777,36.24893 -79.46374,36.24866 -79.45847,36.24853 -79.4449,36.24823 -79.4382,36.24809 -79.42713,36.24787 -79.40473,36.24741 -79.3796,

As you already know, this is a set of **longitude (x)** and **latitude (y)** in a string format. Therefore, we need a little bit of **string operation**.

In [4]:
# make a copy and extract interested columns
geo = df.copy()[['County Name', 'geometry']]

# extract a set of longitude (x) and latitude (y)
geo['x'] = geo.apply(lambda row: [float(i.split(',')[0]) for i in row.geometry[51:-55].split(' ')], axis=1)
geo['y'] = geo.apply(lambda row: [float(i.split(',')[1]) for i in row.geometry[51:-55].split(' ')], axis=1)

# rename and extract interested the column
geo.columns = ['County', 'geometry', 'x', 'y']
geo = geo[['County', 'x', 'y']]

# take a look
geo.head()

,County,x,y
0,Alamance,"[-79.54207, -79.54208, -79.54203, -79.54201, -...","[35.88714, 35.8894, 35.8936, 35.89584, 35.8996..."
1,Alexander,"[-81.00496, -81.01959, -81.02711, -81.0347, -8...","[35.95938, 35.9189, 35.89768, 35.87658, 35.859..."
2,Alleghany,"[-81.32957, -81.34575, -81.35109, -81.35225, -...","[36.51886, 36.5373, 36.54756, 36.56222, 36.568..."
3,Anson,"[-80.21834, -80.2061, -80.19945, -80.19378, -8...","[35.15575, 35.15929, 35.16674, 35.15978, 35.15..."
4,Ashe,"[-81.31908, -81.33849, -81.3456, -81.34906, -8...","[36.37485, 36.36654, 36.35553, 36.35919, 36.33..."


At this point, each **county** has its own **x** and **y**, we have the desired data format to draw a map!!

### Drawing an interactive map

In [5]:
# create ColumnDataSource
source = ColumnDataSource(geo)

# hover tool
hover = HoverTool(tooltips= [('County Name', '@County'),
                             ('(Long, Lat)', '($x, $y)')],
                  mode='mouse')

# create a figure
p = figure(title="Counties in North Carolina", plot_width=650, plot_height=300,
           tools=[hover, 'reset', 'save'], toolbar_location="above",
           x_axis_location=None, y_axis_location=None)

# remove grids
p.grid.grid_line_color = None

# plot boundary
p.patches('x', 'y', source=source, line_color='black', line_width=0.8, fill_alpha=0.4)

# output html file and show it
output_file('county_NC.html')
show(p)

## Part 2: Cattle data
Now let's add one more data on our interactive graph above.

In [6]:
# read cattle data 
df = pd.read_csv('cattle_sold_NC_2012.csv')

# extracted interested columns
df = df[['Year', 'County', 'Data Item', 'Domain Category', 'Value']]

# take a look
df.head(17)

,Year,County,Data Item,Domain Category,Value
0,2012,CRAVEN,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)",(D)
1,2012,JOHNSTON,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)","96,000"
2,2012,LENOIR,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)",(D)
3,2012,PITT,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)","14,000"
4,2012,WAYNE,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)","339,000"
5,2012,WILSON,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)",(D)
6,2012,ALEXANDER,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)","439,000"
7,2012,CATAWBA,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)","122,000"
8,2012,CHATHAM,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)","667,000"
9,2012,DAVIDSON,"CATTLE, INCL CALVES - SALES, MEASURED IN $","OPERATORS, PRINCIPAL: (GENDER = FEMALE)",(D)


From the abvoe table, the cattle sold in USD is mentioned in the **Value** column, and it looks like some counties have **missing values** indicated by **(D)**. In addition, for **WAKE** county (15th, 16th row), we can see that it has more than one value in cattle sold. For counties with multiple values in $, we sum of o. Below it is a little bit of data cleaning.

### Data cleaning

In [7]:
# copy original df
cattle = df.copy()

# specify missing values and remove them
cattle.Value = cattle.Value.str.lstrip().replace('(D)', np.nan)
cattle = cattle.dropna(axis=0)

# convert str to float in 'Value' column
cattle.Value = cattle.Value.apply(lambda row: np.int(''.join(row.split(','))))

# convert county name to 'title' from 'uppercase'
cattle.County = cattle.County.str.title()

# deal with counties who have multiple values in $
cattle = cattle.groupby(['County'])[['Value']].sum()

# take a look
cattle.head()

,Value
County,
Alamance,199000
Alexander,439000
Anson,110000
Ashe,450000
Bladen,1008000


### Merging two dataFrame

In [8]:
# merge geo data and cattle sell
nc = pd.merge(geo, cattle, left_on='County', right_index=True, how='left')

# take a look
nc.head()

,County,x,y,Value
0,Alamance,"[-79.54207, -79.54208, -79.54203, -79.54201, -...","[35.88714, 35.8894, 35.8936, 35.89584, 35.8996...",199000.0
1,Alexander,"[-81.00496, -81.01959, -81.02711, -81.0347, -8...","[35.95938, 35.9189, 35.89768, 35.87658, 35.859...",439000.0
2,Alleghany,"[-81.32957, -81.34575, -81.35109, -81.35225, -...","[36.51886, 36.5373, 36.54756, 36.56222, 36.568...",NaN
3,Anson,"[-80.21834, -80.2061, -80.19945, -80.19378, -8...","[35.15575, 35.15929, 35.16674, 35.15978, 35.15...",110000.0
4,Ashe,"[-81.31908, -81.33849, -81.3456, -81.34906, -8...","[36.37485, 36.36654, 36.35553, 36.35919, 36.33...",450000.0


### Drawing an interactive map
Great!! Let's plot an interactive graph again.

In [9]:
# create color mapper
RdYlBu5.reverse()
mapper = LogColorMapper(palette=RdYlBu5)

# create ColumnDataSource
source = ColumnDataSource(nc)

# hover tool
hover = HoverTool(tooltips= [("County Name", "@County"),
                             ('Cattle Sold', '$@Value')],
                  mode='mouse')

p = figure(title="Cattle Sold in 2012 in North Carolina",
           tools=[hover, 'reset', 'save'], toolbar_location="above",
           x_axis_location=None, y_axis_location=None, plot_width=650, plot_height=300)

p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          line_color="black", line_width=0.8,
          fill_alpha=0.4, fill_color={'field': 'Value', 'transform': mapper},)

output_file('cattle_sold_NC_2012.html')
show(p)